In [1]:
#Загружаем нужные библиотеки

import pandas as pd

import re

from time import sleep

In [2]:
from tqdm import tqdm_notebook

In [3]:
from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.common.exceptions import NoSuchElementException

In [4]:
# подгружаем Хром

driver = Chrome('C:/work/chromedriver/chromedriver.exe')

Собственно грибы растут в 3-х категориях:

1) Для дома и дачи — Продукты питания

2) Для дома и дачи — Растения

3) Личные вещи — Красота и здоровье

4) Предложение услуг

Также по запросам Мухомор и Чага во всех категориях.

Ссылка содержит домен, регион, категорию, указание на страницу и хвост с запросом.

Для первой части обрезаю ссылку до указания страницы "?cd=1&p="

вторая часть — запрос "&q=грибы"

номер страницы подставляю циклом.

In [5]:
# разбиваю ссылку на 2 части

# Продукты питания
url_part_1 = 'https://www.avito.ru/rossiya/produkty_pitaniya?cd=1&p='

# Растения
#url_part_1 = 'https://www.avito.ru/rossiya/rasteniya?cd=1&p='

# Красота и здоровье:
#url_part_1 = 'https://www.avito.ru/rossiya/krasota_i_zdorove/kupit-bad-ASgBAgICAUSEAri~Ag?cd=1&p='

# Красота и здоровье (медицинские изделия):
#url_part_1 = 'https://www.avito.ru/rossiya/krasota_i_zdorove/kupit-meditsinskie_izdeliya-ASgBAgICAUSEAqgJ?cd=1&p='

# Предложение услуг
#url_part_1  = 'https://www.avito.ru/rossiya/predlozheniya_uslug?cd=1&p='

# Другие грибы — первая часть ссылки — Продукты питания

# вторая часть — одинаковая для всех
#url_part_2 = '&q=грибы'

# вторая часть для мухоморов и чаги
#url_part_2 = '&q=мухомор'

#url_part_2 = '&q=чага'

#url_part_2 = '&q=трутовик'

# вторая часть — одинаковая для всех
#url_part_2 = '&q=опенок'

#url_part_2 = '&q=подосиновик'

#url_part_2 = '&q=масленок'

#url_part_2 = '&q=подберезовик'

#url_part_2 = '&q=лисичка'

#url_part_2 = '&q=рыжик'

#url_part_2 = '&q=груздь'

#url_part_2 = '&q=вешенка'

url_part_2 = '&q=волнушка'


In [6]:
# Открываю страницу, чтобы посмотреть, сколько страниц

driver.get(url_part_1 +str(1) +url_part_2)

In [7]:
%%time

links = []


# количество страниц для каждой категории нужно указать вручную

for p in tqdm_notebook(range(1, 4)): # цикл для перехода по страницам, количество страниц нужно указать вручную
    
    url = url_part_1 +str(p) +url_part_2 # формирую ссылку на страницу с объявлениями
    driver.get(url) # открываю эту страницу
    sleep(3) # делаю паузу, даю прогрузиться
    
    # нахожу ссылки на объявления на странице
    urls_list = driver.find_elements_by_class_name('iva-item-sliderLink-2hFV_')
    
    
    for ads_url in urls_list:
        links += [ads_url.get_attribute('href')]  
    


Wall time: 25 s


In [8]:
# проверяю ссылки

links[:5]

['https://www.avito.ru/arhangelsk/produkty_pitaniya/volnushka_1384887866',
 'https://www.avito.ru/uhta/produkty_pitaniya/gruzdi_i_volnushki_1804972271',
 'https://www.avito.ru/berezniki/produkty_pitaniya/prodam_volnushki_2052251750',
 'https://www.avito.ru/kotlas/produkty_pitaniya/volnushki_1977870555',
 'https://www.avito.ru/arhangelsk/produkty_pitaniya/volnushki_2007254828']

In [9]:
len(links)

123

In [10]:
%%time

# список для данных
data = []


for url in tqdm_notebook(links):
    
    driver.get(url)
    sleep(5)

   
    try:
        topic = driver.find_element_by_class_name('title-info-title-text').text
        
    except:
        topic = 'нет'
        
    
    try:
        seller = driver.find_element_by_class_name('seller-info-name').text
        
    except:
        seller = 'нет'
        
    
    try:
        city = driver.find_element_by_class_name('item-address__string').text
    
    except:
        city = 'нет'
    
    
    try:
        ads_text = driver.find_element_by_class_name('item-description-text').text
        
    except:
        ads_text = 'нет'
            
    try:
        price = driver.find_element_by_xpath('/html/body/div[3]/div[1]/div[3]/div[4]/div[2]/div[1]/div/div[1]/div/div[1]/div/div/span/span[1]').text
        
    except:
        price = 'нет'
        
    try:
        user_url = driver.find_element_by_class_name('seller-info-name.js-seller-info-name').find_element_by_tag_name('a').get_attribute('href')
    
    except:
        user_url = 'нет'
    
    
    data.append([topic, city, ads_text, price, seller, user_url, url])  
    


Wall time: 19min 18s


In [11]:
# преобразовываю в датафрейм

head = ['topic', 'city', 'ads_text', 'price', 'seller', 'user_id', 'url']
df = pd.DataFrame(data, columns=head)

In [12]:
df.shape

(123, 7)

In [13]:
df.sample(3)

,topic,city,ads_text,price,seller,user_id,url
95,"Грибы соленые, маринованные,сухие домашние","Санкт-Петербург, пр-т Энергетиков, 31к2","Грибочки домашние.\nСоленые,маринованные,сухие...",300,Частное лицо не Компания,https://www.avito.ru/user/6eadab818976e62b33fa...,https://www.avito.ru/sankt-peterburg/produkty_...
51,"Ягоды замороженные, грибы соленые","Ленинградская область, Выборгский р-н, Выборгс...",Ягоды брусники замороженные сразу после сбора....,300,Валентина,https://www.avito.ru/user/f0a69eaf1708715f6dc3...,https://www.avito.ru/vyborg/produkty_pitaniya/...
67,Грибы,"Ленинградская область, Выборгский р-н, Приморс...","Продам грибы.\nСухие, маринованные, соленые.\n...",нет,Елена,https://www.avito.ru/user/8e4fff7d715e89dbfe93...,https://www.avito.ru/leningradskaya_oblast_pri...


In [14]:
df.to_excel('C:/00_Projects/Rus_mushrooms/Parsing_results/Avito_2020_12_08/mushrooms_16.xlsx', encoding='utf8', index=False)